# Import Libraries

In [0]:
from IPython.display import clear_output
# Import necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

path = "/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/InsideAirbnb/data/"
calendar = pd.read_csv(path+"calendar.csv")
listings = pd.read_csv(path+"listings.csv")
reviews = pd.read_csv(path+"reviews.csv")
clear_output()

# Column lists

In [0]:
col_list = list(listings.columns)
# basicly the text information about the host
# text_columns=['summary','space','description',"neighborhood_overview","notes","transit","access","interaction","house_rules","host_about"]
# 
text_columns=['summary','space',"neighborhood_overview","notes","transit","access","interaction","house_rules","host_about"]
# 
host_string_columns = ["host_name","host_since","amenities"]

host_location_columns = ["host_neighbourhood","street","neighbourhood","neighbourhood_cleansed","city","market","host_location",
                         "latitude","longitude"]

host_numeric_columns = ["host_id","host_acceptance_rate","host_response_time","host_response_rate","host_listings_count",
                        "host_total_listings_count","square_feet","price","weekly_price","monthly_price",
                        "security_deposit","cleaning_fee","guests_included","extra_people","number_of_reviews",
                        "number_of_reviews_ltm","calculated_host_listings_count","calculated_host_listings_count_entire_homes",
                        "calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms",
                        "reviews_per_month"]

host_date_columns = ["first_review"]

host_nights_columns = col_list[67:75]

host_category_columns = ["host_is_superhost","host_has_profile_pic","host_identity_verified","property_type","room_type",
                         "instant_bookable","is_business_travel_ready","require_guest_profile_picture","require_guest_phone_verification"]

host_score_columns = ["review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin",
                      "review_scores_communication","review_scores_location","review_scores_value"
                      ]
host_other_columns = ["host_verifications",]

In [0]:
listings.head(2)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,44077,https://www.airbnb.com/rooms/44077,20200321154917,2020-03-21,cosy comfortable Irish cottage twin,Our house was built in 1937 when there was ple...,"Hi, My name is Teresa and we have 3 rooms ava...",Our house was built in 1937 when there was ple...,none,I like our neighbourhood as there is no shorta...,Dundrum was always known as a Spa Town and the...,There is a bus stop 2 mins up the road from us...,We have a beautiful landscaped garden with tab...,We live on the premises so we are always on ha...,Welcome to our cottage and we hope you will en...,NaN,NaN,https://a0.muscache.com/im/pictures/525706/050...,NaN,193005,https://www.airbnb.com/users/show/193005,Teresa,2010-08-06,Dublin Ireland,"I am a friendly outgoing Irishwoman ,who love...",within an hour,100%,100%,t,https://a0.muscache.com/im/users/193005/profil...,https://a0.muscache.com/im/users/193005/profil...,Churchtown,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,"Churchtown, Ireland",Churchtown,Dn Laoghaire-Rathdown,...,$0.00,3,1125,3,3,1125,1125,3.0,1125.0,a week ago,t,7,11,23,275,2020-03-21,227,38,2011-03-20,2020-03-06,95.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,NaN,t,f,moderate,t,t,3,0,3,0,2.07
1,85148,https://www.airbnb.com/rooms/85148,20200321154917,2020-03-21,Habitación en Dublín Double room,NaN,I'm renting a double room in my house in the s...,I'm renting a double room in my house in the s...,none,NaN,NaN,NaN,NaN,NaN,"Sorry, No smoking or pets. Lo siento esta proh...",NaN,NaN,https://a0.muscache.com/im/pictures/583752/61c...,NaN,466111,https://www.airbnb.com/users/show/466111,Don,2011-03-27,"Dublin, Ireland",I am single and live alone here so when I'm ...,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/466111/profil...,https://a0.muscache.com/im/users/466111/profil...,Templeogue,1.0,1.0,"['email', 'phone', 'reviews']",t,f,"Templeogue, Dublin, Ireland",Templeogue,South Dublin,...,$10.00,1,5,1,1,5,5,1.0,5.0,4 months ago,t,0,0,0,140,2020-03-21,1,1,2019-05-11,2019-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,f,flexible,f,f,1,0,1,0,0.09


In [0]:
df_text = listings[['id']+text_columns]
df_text.sample(1).T

,3258
id,18846978
summary,"Spacious, welcoming large detached home in up..."
space,"There is a large family sitting room, formal d..."
neighborhood_overview,Castleknock is a quiet neighbourhood with a go...
notes,NaN
transit,The city centre is twenty minutes away and wel...
access,The house is detached and in a residential are...
interaction,I am available to provide guidance and informa...
house_rules,Infants are welcome provided guests can provid...
host_about,A Dubliner who loves her city and wants to sha...


In [0]:
df_numeric = listings[['id']+host_numeric_columns]
df_numeric.head(1).T

,0
id,44077
host_id,193005
host_acceptance_rate,100%
host_response_time,within an hour
host_response_rate,100%
host_listings_count,3
host_total_listings_count,3
square_feet,NaN
price,$50.00
weekly_price,NaN


In [0]:
df_score = listings[['id']+host_score_columns]
df_score.head(1).T

,0
id,44077.0
review_scores_rating,95.0
review_scores_accuracy,10.0
review_scores_cleanliness,10.0
review_scores_checkin,10.0
review_scores_communication,10.0
review_scores_location,10.0
review_scores_value,10.0


# combine text together

In [0]:
df_text.fillna(' ',inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [0]:
df_text['combined'] = df_text['summary']
for target in text_columns:
  df_text['combined'] = df_text['combined']+df_text[target]
df_text_combined = df_text[['id','combined']]
df_text_combined.head(1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,combined
0,44077,Our house was built in 1937 when there was ple...


# Pre-Processing 
## for one column

In [0]:
target = text_columns[0]

In [0]:
df = df_text[['id',target]]

In [0]:
df.head(2)

,id,summary
0,44077,Our house was built in 1937 when there was ple...
1,85148,


## Lowercasing

In [0]:
# df[target] = df[target].apply(Lower)
df[target] = df[target].str.lower()
clear_output()
df.head(3)

,id,summary
0,44077,our house was built in 1937 when there was ple...
1,85148,
2,85156,our cottage is a charming light filled cottage...


## Removing Punctuation

In [0]:
df[target] = df[target].str.replace('[^\w\s]','')
clear_output()

In [0]:
df.head(3)

,id,summary
0,44077,our house was built in 1937 when there was ple...
1,85148,
2,85156,our cottage is a charming light filled cottage...


## Word Token
split sentence in to a list of word

In [0]:
text = df[target].iloc[0]
text

'our house was built in 1937 when there was plenty of land originally dundrum was considered a spa town because of it recuperative powers it is only 10 kilometres  to the mountains yet we are only 6 kilometres to the city centre  dublin is a small compact city  our guests tell us they sleep like logs we do provide good mattresses too though   when you are a guest at our house we recognise that this is your holiday so we endeavour to do the best we can to make it as memorable as possible '

In [0]:
#@title split sentence in to a list of word
from nltk.tokenize import word_tokenize
token = word_tokenize(text)
token[0:5]

['our', 'house', 'was', 'built', 'in']

In [0]:
#@title combine a list of word to a sentence
token_sentence = " ".join(token)
token_sentence 

'our house was built in 1937 when there was plenty of land originally dundrum was considered a spa town because of it recuperative powers it is only 10 kilometres to the mountains yet we are only 6 kilometres to the city centre dublin is a small compact city our guests tell us they sleep like logs we do provide good mattresses too though when you are a guest at our house we recognise that this is your holiday so we endeavour to do the best we can to make it as memorable as possible'

## Removing Stop-Words & unsual Characters

In [0]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
stops = set(stopwords.words('english'))
clear_output()

In [0]:
def remove_stopwords(text):
  if(type(text)==type(1.0)):
    return np.NaN
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  token = word_tokenize(text)
  r_stopwords = [x for x in token if x not in stops] # means remove stop words
  sentence = " ".join(r_stopwords)
  return sentence

In [0]:
remove_stopwords(text)

In [0]:
df[target+'_rstop'] = df[target].apply(remove_stopwords)
clear_output()

In [0]:
df.head()

,id,summary,summary_rstop
0,44077,our house was built in 1937 when there was ple...,house built 1937 plenty land originally dundru...
1,85148,,
2,85156,our cottage is a charming light filled cottage...,cottage charming light filled cottage vibrant ...
3,121030,welcoming victorian house breakfast included d...,welcoming victorian house breakfast included d...
4,159889,washing can be done at a cost of five euro per...,washing done cost five euro per load light bre...


# Common/Rare Words

In [0]:
#@title Common Words
df_dn = df[target+'_rstop'].dropna()
df_freq = pd.Series(' '.join(df_dn).split())
print("count of unique words {}\n".format(len(df_freq.value_counts())))
print("Top 10 words")
FreqSeries = df_freq.value_counts()
FreqSeries[0:10]

count of unique words 11689

Top 10 words


city         6772
dublin       6347
walk         5218
apartment    4497
centre       4496
room         3710
minutes      3507
house        3205
bus          2967
bedroom      2746
dtype: int64

In [0]:
FreqSeries.values>1500

array([ True,  True,  True, ..., False, False, False])

In [0]:
onlyWords = list(FreqSeries[FreqSeries.values==1].index)
freqWords = list(FreqSeries[FreqSeries.values>2000].index)

In [0]:
of = onlyWords+freqWords
len(of)

5468

In [0]:
# df[target+'_rOnly'] = df[target].apply(lambda x: " ".join(x for x in x.split() if x not in onlyWords))

In [0]:
def getOnly_FreqWords(df,target):
  df_dn = df[target].dropna()
  df_freq = pd.Series(' '.join(df_dn).split())
  FreqSeries = df_freq.value_counts()
  onlyWords = list(FreqSeries[FreqSeries.values==1].index)
  freqWords = list(FreqSeries[FreqSeries.values>2000].index)
  return onlyWords+freqWords

In [0]:
print("Low 10 words")
df_freq.value_counts()[-10:]

Low 10 words


snuggling      1
menities       1
segways        1
merges         1
wines          1
kilmainhaim    1
refurnish      1
hesitation     1
keles          1
vegetarians    1
dtype: int64

# Spelling Correction
takes long time, cannot work on our dataset


In [0]:
!pip install textblob
!python -m textblob.download_corpora
clear_output()
from textblob import TextBlob

In [0]:
def correct_spelling(x):
  if type(x)==np.float:
    return np.NaN
  else:
    return str(TextBlob(x).correct())

In [0]:
# df[target+'_rstop'] = df[target+'_rstop'].apply(correct_spelling)

# Stemming 保留词干
Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving the goal of creating a standard word, and often includes the removal of derivational affixes.

In [0]:
from nltk.stem import PorterStemmer
pst = PorterStemmer()

In [0]:
pst.stem('playing') # example of stemming

'play'

In [0]:
# stemming("playing")

In [0]:
def stemming(text):
  if(type(text)==type(1.0)):
    return np.NaN
  text_list = [pst.stem(word) for word in text.split()]
  text_res = ' '.join(text_list)
  return text_res

In [0]:
df[target+'_rstop']

0       house built 1937 plenty land originally dundru...
1                                                        
2       cottage charming light filled cottage vibrant ...
3       welcoming victorian house breakfast included d...
4       washing done cost five euro per load light bre...
                              ...                        
9138    tastefully decorated 3 bedroom apartment heart...
9139    fully equipped 1 bedroom apartment oconnell br...
9140              spacious 3 bedroom apartment temple bar
9141                    fully equipped apartment dublin 2
9142                                                     
Name: summary_rstop, Length: 9143, dtype: object

In [0]:
df[target+'_stem'] = df[target+'_rstop'].apply(stemming)
clear_output()
df.head()

,id,summary,summary_rstop,summary_stem
0,44077,our house was built in 1937 when there was ple...,house built 1937 plenty land originally dundru...,hous built 1937 plenti land origin dundrum con...
1,85148,,,
2,85156,our cottage is a charming light filled cottage...,cottage charming light filled cottage vibrant ...,cottag charm light fill cottag vibrant distric...
3,121030,welcoming victorian house breakfast included d...,welcoming victorian house breakfast included d...,welcom victorian hous breakfast includ doubl b...
4,159889,washing can be done at a cost of five euro per...,washing done cost five euro per load light bre...,wash done cost five euro per load light breakf...


according to Andrew's Colab, Lemmatization is more popular than Stemming

# Lemmatization 词形还原
Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma . It is a more effective option than stemming because it converts the word into its root word, rather than just stripping the suffices. It makes use of the vocabulary and does a morphological analysis to obtain the root word. Therefore, lemmatization is usually preferred over stemming.

In [0]:
!pip install textblob
!python -m textblob.download_corpora
from textblob import Word
clear_output()

In [0]:
def lemmatization(text):
  if(type(text)==type(1.0)):
    return np.NaN
  text_list = [Word(word).lemmatize() for word in text.split()]
  text_res = ' '.join(text_list)
  return text_res

In [0]:
lemmatization("we can see it transform fully into fulli according to Andrew's Colab, Lemmatization is more popular than Stemming")

"we can see it transform fully into fulli according to Andrew's Colab, Lemmatization is more popular than Stemming"

In [0]:
# df[target+'_lemm'] = df[target+'_rstop'].apply(lemmatization)
# clear_output()
# df.head()

# Apply on dataset

In [0]:
save_path = '/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/DataMining_Assignment/Data/'
# for target in text_columns:
#   df = listings[['id','host_id',target]]
#   df.fillna('',inplace=True)
#   df_res = preProcessing(df)
#   df_res.to_csv(save_path+"PP_"+target+'.csv')
#   clear_output()
# df_res.head()

In [0]:
def preProcessing(df):
  df[target] = df[target].str.lower()
  df[target] = df[target].str.replace('[^\w\s]','')
  df['text'] = df[target]
  df['rstop'] = df[target].apply(remove_stopwords)
  # df['stemming'] = df['rstop'].apply(stemming)
  df['lemmatization'] = df['rstop'].apply(lemmatization)
  # df['lemmatization'] = df['lemmatization'].apply(lambda x: " ".join(x for x in x.split() if x not in getOnly_FreqWords(df,'lemmatization')))
  df.drop(columns=[target],inplace=True)
  return df

In [0]:
df_text_combined.head(1)

,id,combined
0,44077,Our house was built in 1937 when there was ple...


In [0]:
target='combined'
df_combined = preProcessing(df_text_combined)

In [0]:
df_combined.tail()

,id,text,rstop,lemmatization
9138,42971991,tastefully decorated 3 bedroom apartment in th...,tastefully decorated 3 bedroom apartment heart...,tastefully decorated 3 bedroom apartment heart...
9139,42972454,fully equipped 1 bedroom apartment in oconnell...,fully equipped 1 bedroom apartment oconnell br...,fully equipped 1 bedroom apartment oconnell br...
9140,42972621,spacious 3 bedroom apartment in temple barspac...,spacious 3 bedroom apartment temple barspaciou...,spacious 3 bedroom apartment temple barspaciou...
9141,42972843,fully equipped apartment in dublin 2fully equi...,fully equipped apartment dublin 2fully equippe...,fully equipped apartment dublin 2fully equippe...
9142,42973665,hello we are a decades old family bus...,hello decades old family business catering ten...,hello decade old family business catering tena...


In [0]:
res = getOnly_FreqWords(df_text_combined,'lemmatization')

In [0]:
res

In [0]:
df_combined.to_csv(save_path+'text_combined_lemm.csv')

In [0]:
df_combined['rcommon'] = df_combined['lemmatization'].apply(lambda x: " ".join(x for x in x.split() if x not in res))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df_combined.to_csv(save_path+'text_combined_lemm_rcommon.csv')

In [0]:
df = pd.read_csv(save_path+'text_combined_lemm_rcommon.csv')

In [0]:
df['text'].iloc[0]

'our house was built in 1937 when there was plenty of land originally dundrum was considered a spa town because of it recuperative powers it is only 10 kilometres  to the mountains yet we are only 6 kilometres to the city centre  dublin is a small compact city  our guests tell us they sleep like logs we do provide good mattresses too though   when you are a guest at our house we recognise that this is your holiday so we endeavour to do the best we can to make it as memorable as possible our house was built in 1937 when there was plenty of land originally dundrum was considered a spa town because of it recuperative powers it is only 10 kilometres  to the mountains yet we are only 6 kilometres to the city centre  dublin is a small compact city  our guests tell us they sleep like logs we do provide good mattresses too though   when you are a guest at our house we recognise that this is your holiday so we endeavour to do the best we can to make it as memorable as possible hi my name is ter

In [0]:
df['rcommon'].iloc[0]

'built 1937 plenty land originally dundrum considered spa recuperative power kilometre mountain yet 6 kilometre compact tell sleep log provide mattress though recognise holiday endeavour memorable possible built 1937 plenty land originally dundrum considered spa recuperative power kilometre mountain yet 6 kilometre compact tell sleep log provide mattress though recognise holiday endeavour memorable possible hi name teresa 14 bedroomed cottage situated dundrum mile sized suitable beautifully furnished suitable fireplace tastefully furnished designer owner listing belongs allow contact booking thank supply light morning cereal jam honey toast etc 11am ask prepare hot drink simple snack longterm cooking prior arrangement permissible ask special rate bathi shortage thing plus load supermarket sea mountain far neither 6 kilometre easily accessable explore always known spa air conducive sleep always telling sleep sense peace brings onwards mainline beyond every 35 approx citywe landscaped ta